In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# Imports

import numpy as np
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  """


In [8]:
data = pd.read_csv('/content/gdrive/MyDrive/AirPassengers.csv')
data = data.rename(columns={'#Passengers': 'Passengers'})
data = data["Passengers"]
df = data.values
df = df.astype('float32') 

In [47]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Activation, Flatten
from keras.layers.convolutional_recurrent import ConvLSTM2D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [9]:
scaler = MinMaxScaler()
df = df.reshape(-1, 1)  # reshaping for scaling
scaler = MinMaxScaler() # optional for single feature
df = scaler.fit_transform(df)

In [10]:
train_size = int(len(df) * 0.66)  #seq is imp
test_size = len(df) - train_size
train, test = df[0:train_size,:], df[train_size:len(df),:]
train_size,test_size

(95, 49)

In [14]:
from keras.preprocessing.sequence import TimeseriesGenerator # Generates batches for sequence data
win_size = 10
batch_size = 1
train_generator = TimeseriesGenerator(train,train,length=win_size,batch_size=batch_size)
len(train),len(train_generator)

(95, 85)

In [20]:
validation_generator = TimeseriesGenerator(test, test, length=win_size ,batch_size=batch_size)

In [21]:
num_features = 1  # input

### Simple RNN

In [29]:
print('RNN Model...')
model = Sequential()
model.add(SimpleRNN(64, input_shape=(win_size,num_features), activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])
print(model.summary()) 

RNN Model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 64)                4224      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 4,289
Trainable params: 4,289
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
model.fit(train_generator,verbose=2,epochs=80, validation_data=validation_generator) 

Epoch 1/80
85/85 - 0s - loss: 2.3031e-04 - accuracy: 0.0118 - val_loss: 0.0033 - val_accuracy: 0.0256
Epoch 2/80
85/85 - 0s - loss: 1.9720e-04 - accuracy: 0.0118 - val_loss: 0.0041 - val_accuracy: 0.0256
Epoch 3/80
85/85 - 0s - loss: 1.6235e-04 - accuracy: 0.0118 - val_loss: 0.0052 - val_accuracy: 0.0256
Epoch 4/80
85/85 - 0s - loss: 2.4151e-04 - accuracy: 0.0118 - val_loss: 0.0061 - val_accuracy: 0.0256
Epoch 5/80
85/85 - 0s - loss: 2.6251e-04 - accuracy: 0.0118 - val_loss: 0.0044 - val_accuracy: 0.0256
Epoch 6/80
85/85 - 0s - loss: 3.9206e-04 - accuracy: 0.0118 - val_loss: 0.0037 - val_accuracy: 0.0256
Epoch 7/80
85/85 - 0s - loss: 3.6675e-04 - accuracy: 0.0118 - val_loss: 0.0031 - val_accuracy: 0.0256
Epoch 8/80
85/85 - 0s - loss: 2.8590e-04 - accuracy: 0.0118 - val_loss: 0.0034 - val_accuracy: 0.0256
Epoch 9/80
85/85 - 0s - loss: 2.6399e-04 - accuracy: 0.0118 - val_loss: 0.0024 - val_accuracy: 0.0256
Epoch 10/80
85/85 - 0s - loss: 2.4976e-04 - accuracy: 0.0118 - val_loss: 0.0035 - 

In [36]:
import math

pred_tr = model.predict(train_generator)
pred_te = model.predict(validation_generator)
# Inverse transformation to bring actual values

pred_tr = scaler.inverse_transform(pred_tr)
ytr_act = scaler.inverse_transform(train)
pred_te = scaler.inverse_transform(pred_te)
yte_act = scaler.inverse_transform(test)

train_rmse = math.sqrt(mean_squared_error(ytr_act[win_size:], pred_tr[:,0]))
test_rmse = math.sqrt(mean_squared_error(yte_act[win_size:], pred_te[:,0]))
train_rmse,test_rmse

(3.3483001552651364, 27.149009661236956)

### 

In [38]:
model = Sequential()
model.add(LSTM(50, input_shape=(win_size, num_features)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])
print(model.summary()) 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
model.fit(train_generator,verbose=2,epochs=80, validation_data=validation_generator) 

Epoch 1/80
85/85 - 4s - loss: 0.0177 - accuracy: 0.0118 - val_loss: 0.0268 - val_accuracy: 0.0256
Epoch 2/80
85/85 - 0s - loss: 0.0085 - accuracy: 0.0118 - val_loss: 0.0279 - val_accuracy: 0.0256
Epoch 3/80
85/85 - 0s - loss: 0.0066 - accuracy: 0.0118 - val_loss: 0.0254 - val_accuracy: 0.0256
Epoch 4/80
85/85 - 0s - loss: 0.0062 - accuracy: 0.0118 - val_loss: 0.0250 - val_accuracy: 0.0256
Epoch 5/80
85/85 - 0s - loss: 0.0057 - accuracy: 0.0118 - val_loss: 0.0241 - val_accuracy: 0.0256
Epoch 6/80
85/85 - 0s - loss: 0.0056 - accuracy: 0.0118 - val_loss: 0.0236 - val_accuracy: 0.0256
Epoch 7/80
85/85 - 0s - loss: 0.0054 - accuracy: 0.0118 - val_loss: 0.0239 - val_accuracy: 0.0256
Epoch 8/80
85/85 - 0s - loss: 0.0055 - accuracy: 0.0118 - val_loss: 0.0216 - val_accuracy: 0.0256
Epoch 9/80
85/85 - 0s - loss: 0.0049 - accuracy: 0.0118 - val_loss: 0.0255 - val_accuracy: 0.0256
Epoch 10/80
85/85 - 0s - loss: 0.0058 - accuracy: 0.0118 - val_loss: 0.0200 - val_accuracy: 0.0256
Epoch 11/80
85/85 -

In [40]:
pred_tr = model.predict(train_generator)
pred_te = model.predict(validation_generator)
# Inverse transformation to bring actual values

pred_tr = scaler.inverse_transform(pred_tr)
ytr_act = scaler.inverse_transform(train)
pred_te = scaler.inverse_transform(pred_te)
yte_act = scaler.inverse_transform(test)

train_rmse = math.sqrt(mean_squared_error(ytr_act[win_size:], pred_tr[:,0]))
test_rmse = math.sqrt(mean_squared_error(yte_act[win_size:], pred_te[:,0]))
train_rmse,test_rmse

(15.652905744683737, 56.66753455600702)

In [41]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(win_size, num_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 50)            10400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(train_generator,verbose=2,epochs=80, validation_data=validation_generator) 

Epoch 1/80
85/85 - 6s - loss: 0.0176 - val_loss: 0.0473
Epoch 2/80
85/85 - 1s - loss: 0.0073 - val_loss: 0.0274
Epoch 3/80
85/85 - 1s - loss: 0.0080 - val_loss: 0.0350
Epoch 4/80
85/85 - 1s - loss: 0.0070 - val_loss: 0.0293
Epoch 5/80
85/85 - 1s - loss: 0.0060 - val_loss: 0.0260
Epoch 6/80
85/85 - 1s - loss: 0.0060 - val_loss: 0.0272
Epoch 7/80
85/85 - 1s - loss: 0.0056 - val_loss: 0.0274
Epoch 8/80
85/85 - 1s - loss: 0.0064 - val_loss: 0.0270
Epoch 9/80
85/85 - 1s - loss: 0.0060 - val_loss: 0.0415
Epoch 10/80
85/85 - 1s - loss: 0.0060 - val_loss: 0.0257
Epoch 11/80
85/85 - 1s - loss: 0.0067 - val_loss: 0.0253
Epoch 12/80
85/85 - 1s - loss: 0.0059 - val_loss: 0.0287
Epoch 13/80
85/85 - 1s - loss: 0.0062 - val_loss: 0.0249
Epoch 14/80
85/85 - 1s - loss: 0.0063 - val_loss: 0.0326
Epoch 15/80
85/85 - 1s - loss: 0.0056 - val_loss: 0.0273
Epoch 16/80
85/85 - 1s - loss: 0.0057 - val_loss: 0.0239
Epoch 17/80
85/85 - 1s - loss: 0.0057 - val_loss: 0.0515
Epoch 18/80
85/85 - 1s - loss: 0.0066 - 

In [43]:
pred_tr = model.predict(train_generator)
pred_te = model.predict(validation_generator)
# Inverse transformation to bring actual values

pred_tr = scaler.inverse_transform(pred_tr)
ytr_act = scaler.inverse_transform(train)
pred_te = scaler.inverse_transform(pred_te)
yte_act = scaler.inverse_transform(test)

train_rmse = math.sqrt(mean_squared_error(ytr_act[win_size:], pred_tr[:,0]))
test_rmse = math.sqrt(mean_squared_error(yte_act[win_size:], pred_te[:,0]))
train_rmse,test_rmse

(18.636060055038456, 33.520956743739056)